In [1]:
import pickle as pickle
import numpy as np

In [2]:

from numpy.fft import fft
from numpy import zeros, floor, log10, log, mean, array, sqrt, vstack, cumsum,ones, log2, std
from numpy.linalg import svd, lstsq
import time

In [3]:
sub=pickle.load(open('/content/drive/My Drive/Colab Notebooks/s02.dat', 'rb'),encoding="latin1")
print(sub)



{'labels': array([[9.  , 5.03, 7.13, 6.62],
       [8.01, 7.1 , 8.04, 8.08],
       [9.  , 9.  , 8.96, 8.05],
       [6.05, 1.  , 5.04, 7.03],
       [5.04, 3.  , 3.65, 5.04],
       [5.  , 4.94, 5.04, 1.01],
       [4.96, 1.99, 4.08, 1.  ],
       [9.  , 9.  , 9.  , 9.  ],
       [9.  , 7.  , 9.  , 7.18],
       [4.99, 1.  , 9.  , 1.  ],
       [7.08, 1.  , 7.01, 7.  ],
       [8.01, 6.06, 7.12, 9.  ],
       [9.  , 4.99, 9.  , 9.  ],
       [8.94, 9.  , 8.97, 9.  ],
       [9.  , 8.01, 8.06, 9.  ],
       [6.  , 5.05, 8.05, 6.14],
       [9.  , 3.  , 6.03, 7.41],
       [9.  , 6.15, 9.  , 8.08],
       [9.  , 8.32, 8.06, 6.99],
       [8.97, 9.  , 8.03, 9.  ],
       [1.  , 1.  , 5.04, 1.  ],
       [5.  , 1.  , 4.97, 5.  ],
       [1.  , 2.97, 1.  , 7.82],
       [7.08, 9.  , 6.06, 9.  ],
       [9.  , 9.  , 8.1 , 9.  ],
       [9.  , 8.04, 9.  , 9.  ],
       [4.96, 5.01, 3.01, 5.05],
       [4.06, 4.97, 7.14, 7.04],
       [2.99, 7.04, 7.03, 9.  ],
       [2.01, 7.05, 1.  , 7.13],

In [4]:

df=sub["data"]
print(df.shape)

(40, 40, 8064)


In [ ]:
df[0]

array([[-1.11163104e+01,  7.18124410e+01,  1.22849624e+02, ...,
        -5.89884788e+01, -4.73678399e+01, -2.12020108e+01],
       [ 1.75659861e+01, -1.53853133e+02, -2.53610232e+02, ...,
         1.03480282e+02,  1.06308826e+02,  8.30637609e+01],
       [ 8.79215594e+00,  1.96108361e+01,  1.63720087e+01, ...,
        -9.66732098e+00, -9.45398407e+00, -7.48393557e+00],
       ...,
       [-2.98172952e+02, -3.22110407e+02, -3.18329164e+02, ...,
        -1.72957656e+03, -1.72938906e+03, -1.73557655e+03],
       [ 3.85814397e+03,  3.77757879e+03,  3.71361492e+03, ...,
         8.82584191e+03,  8.69547279e+03,  8.67105910e+03],
       [ 1.61612512e-02,  1.61612512e-02,  1.61612512e-02, ...,
        -5.48378389e-02, -5.38378517e-02, -5.48378389e-02]])

In [5]:
label=sub['labels']
label

array([[9.  , 5.03, 7.13, 6.62],
       [8.01, 7.1 , 8.04, 8.08],
       [9.  , 9.  , 8.96, 8.05],
       [6.05, 1.  , 5.04, 7.03],
       [5.04, 3.  , 3.65, 5.04],
       [5.  , 4.94, 5.04, 1.01],
       [4.96, 1.99, 4.08, 1.  ],
       [9.  , 9.  , 9.  , 9.  ],
       [9.  , 7.  , 9.  , 7.18],
       [4.99, 1.  , 9.  , 1.  ],
       [7.08, 1.  , 7.01, 7.  ],
       [8.01, 6.06, 7.12, 9.  ],
       [9.  , 4.99, 9.  , 9.  ],
       [8.94, 9.  , 8.97, 9.  ],
       [9.  , 8.01, 8.06, 9.  ],
       [6.  , 5.05, 8.05, 6.14],
       [9.  , 3.  , 6.03, 7.41],
       [9.  , 6.15, 9.  , 8.08],
       [9.  , 8.32, 8.06, 6.99],
       [8.97, 9.  , 8.03, 9.  ],
       [1.  , 1.  , 5.04, 1.  ],
       [5.  , 1.  , 4.97, 5.  ],
       [1.  , 2.97, 1.  , 7.82],
       [7.08, 9.  , 6.06, 9.  ],
       [9.  , 9.  , 8.1 , 9.  ],
       [9.  , 8.04, 9.  , 9.  ],
       [4.96, 5.01, 3.01, 5.05],
       [4.06, 4.97, 7.14, 7.04],
       [2.99, 7.04, 7.03, 9.  ],
       [2.01, 7.05, 1.  , 7.13],
       [4.

In [6]:


channel = [1,2,3,4,6,11,13,17,19,20,21,25,29,31] #14 Channels chosen to fit Emotiv Epoch+
band = [4,8,12,16,25,45] #5 bands
window_size = 256 #Averaging band power of 2 sec
step_size = 16 #Each 0.125 sec update once
sample_rate = 128 #Sampling rate of 128 Hz
subjectList = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32']

In [7]:
def FFT_Processing (sub, channel, band, window_size, step_size, sample_rate):
    '''
    arguments:  string subject
                list channel indice
                list band
                int window size for FFT
                int step size for FFT
                int sample rate for FFT
    return:     void
    '''
    meta = []
    with open('/content/drive/My Drive/Colab Notebooks/s' + sub + '.dat', 'rb') as file:

        subject = pickle.load(file, encoding='latin1') #resolve the python 2 data problem by encoding : latin1

        for i in range (0,40):
            # loop over 0-39 trails
            data = subject["data"][i]
            labels = subject["labels"][i]
            start = 0;

            while start + window_size < data.shape[1]:
                meta_array = []
                meta_data = [] #meta vector for analysis
                for j in channel:
                    X = data[j][start : start + window_size] #Slice raw data over 2 sec, at interval of 0.125 sec
                    Y = bin_power(X, band, sample_rate) #FFT over 2 sec of channel j, in seq of theta, alpha, low beta, high beta, gamma
                    meta_data = meta_data + list(Y[0])

                meta_array.append(np.array(meta_data))
                meta_array.append(labels)

                meta.append(np.array(meta_array))    
                start = start + step_size
                
        meta = np.array(meta)
        np.save('/content/drive/My Drive/Colab Notebooks/pre' + sub, meta, allow_pickle=True, fix_imports=True)

In [ ]:

FFT_Processing ("02", channel, band, window_size, step_size, sample_rate)

NameError: ignored

In [8]:

def bin_power(X,Band,Fs):

    C = fft(X)
    C = abs(C)
    Power =zeros(len(Band)-1);
    for Freq_Index in range(0,len(Band)-1):
        Freq = float(Band[Freq_Index])                                        ## Xin Liu
        Next_Freq = float(Band[Freq_Index+1])
        Power[Freq_Index] = sum(C[int(Freq/Fs*len(X)):int(Next_Freq/Fs*len(X))])
    Power_Ratio = Power/sum(Power)
    return Power, Power_Ratio    


In [14]:
bin_power(df[1][1][:256], band, sample_rate)

(array([40854.05256639, 10072.83963647,  3934.23940947,  4836.23233583,
        10666.0218218 ]),
 array([0.58061522, 0.14315456, 0.05591316, 0.06873223, 0.15158483]))

In [11]:
ff=ff/sum(ff)

In [12]:
ff

array([0.58061522, 0.14315456, 0.05591316, 0.06873223, 0.15158483])